In [2]:
import geopandas as gpd
import pandas as pd
from pathlib import Path

In [3]:
# get tigerline / census shapefiles from year 2020
dir = Path("shp/")
tl_shps = list(dir.glob("**/tl*20.shp"))
tl_shps

[PosixPath('shp/tl_2020_02_county20.shp'),
 PosixPath('shp/tl_2020_02_tract20.shp'),
 PosixPath('shp/tl_2020_02_place20.shp')]

In [4]:
# read in results table and create geodataframes, reprojecting to web mercator
crs = 3857
results = pd.read_csv("tbl/data_to_export.csv")
county = gpd.read_file(tl_shps[0]).to_crs(crs)
tract = gpd.read_file(tl_shps[1]).to_crs(crs)
place = gpd.read_file(tl_shps[2]).to_crs(crs)

In [5]:
# note that results GEOID column uses 3 digit county code, 5 digit place code, and 9 digit tract code
# so we will need to adjust the tract ID (removing the first 2 digits) to merge the geometry to the results
results["GEOID"].unique()

array(['013', '020', '050', '060', '063', '066', '068', '070', '090',
       '100', '105', '110', '122', '130', '150', '158', '164', '170',
       '180', '185', '188', '195', '198', '220', '230', '240', '275',
       '282', '290', '020000101', '020000102', '020002900', '090980000',
       '00650', '00760', '00870', '01090', '01200', '01305', '01390',
       '01420', '01560', '01860', '01970', '02080', '03110', '03220',
       '03440', '03550', '03880', '03990', '04210', '04430', '04500',
       '04670', '05000', '05585', '05750', '06245', '06520', '06630',
       '06850', '07070', '07620', '08740', '09600', '09657', '09710',
       '10150', '11690', '11800', '12350', '12680', '12920', '12970',
       '13230', '13340', '13450', '13670', '13780', '13860', '13890',
       '14000', '14110', '14330', '14880', '15320', '15430', '16360',
       '16420', '16530', '16630', '16750', '17190', '17300', '17410',
       '17670', '17850', '17960', '18510', '18620', '18675', '18805',
       '18925', '

In [6]:
# prep county gdf, renaming to GEOID and dropping all columns except geometry
county["GEOID"] = county["COUNTYFP20"]
county = county[["GEOID", "geometry"]]
county

,GEOID,geometry
0,050,"MULTIPOLYGON (((-17838823.054 8815510.630, -17..."
1,164,"POLYGON ((-17800003.164 7657440.452, -17797722..."
2,060,"POLYGON ((-17588474.981 8103444.343, -17586660..."
3,070,"MULTIPOLYGON (((-17963061.617 8125781.782, -17..."
4,170,"POLYGON ((-17032091.483 9033337.513, -17032051..."
5,013,"MULTIPOLYGON (((-18371741.628 7280122.270, -18..."
6,158,"POLYGON ((-18510920.602 8765639.205, -18510197..."
7,185,"POLYGON ((-18587209.296 10550351.101, -1858718..."
8,180,"MULTIPOLYGON (((-18727449.034 9775291.850, -18..."
9,016,"MULTIPOLYGON (((-19849132.149 6778748.358, -19..."


In [7]:
# prep tract gdf, using only last 9 digits of GEOID, renaming to GEOID and dropping all columns except geometry
tract["GEOID"] = tract["GEOID20"].str[-9:]
tract = tract[["GEOID", "geometry"]]
tract

,GEOID,geometry
0,130000400,"POLYGON ((-14651810.932 7425976.042, -14651775..."
1,170001004,"POLYGON ((-16632862.497 8761005.812, -16632281..."
2,170000701,"POLYGON ((-16667088.343 8771828.153, -16667084..."
3,170000102,"POLYGON ((-16718098.718 8899729.860, -16717843..."
4,170000604,"POLYGON ((-16732456.150 8689910.990, -16732156..."
...,...,...
172,020000500,"POLYGON ((-16694422.955 8679565.931, -16692482..."
173,198000300,"POLYGON ((-14514793.225 7547112.508, -14514713..."
174,198940100,"POLYGON ((-14661178.690 7381717.416, -14660341..."
175,198000100,"POLYGON ((-14991024.909 7615027.450, -14990943..."


In [8]:
# prep place gdf, renaming to GEOID and dropping all columns except geometry
place["GEOID"] = place["PLACEFP20"]
place = place[["GEOID", "geometry"]]
place

,GEOID,geometry
0,06520,"POLYGON ((-18024480.810 8575001.052, -18024480..."
1,14330,"POLYGON ((-17734409.490 8759711.396, -17734240..."
2,42380,"POLYGON ((-17975083.009 8584489.501, -17973936..."
3,45460,"POLYGON ((-17853235.588 8746199.094, -17851549..."
4,47990,"POLYGON ((-18518903.879 8479783.747, -18518804..."
...,...,...
350,85280,"POLYGON ((-16739340.369 8763503.840, -16739296..."
351,85290,"POLYGON ((-16176806.247 8842804.748, -16176557..."
352,85610,"POLYGON ((-16736277.748 10265967.755, -1673533..."
353,85680,"POLYGON ((-17005373.358 7893332.312, -17004122..."


In [9]:
# combine all polygons into one gdf
polys = pd.concat([county, tract, place])
polys

,GEOID,geometry
0,050,"MULTIPOLYGON (((-17838823.054 8815510.630, -17..."
1,164,"POLYGON ((-17800003.164 7657440.452, -17797722..."
2,060,"POLYGON ((-17588474.981 8103444.343, -17586660..."
3,070,"MULTIPOLYGON (((-17963061.617 8125781.782, -17..."
4,170,"POLYGON ((-17032091.483 9033337.513, -17032051..."
...,...,...
350,85280,"POLYGON ((-16739340.369 8763503.840, -16739296..."
351,85290,"POLYGON ((-16176806.247 8842804.748, -16176557..."
352,85610,"POLYGON ((-16736277.748 10265967.755, -1673533..."
353,85680,"POLYGON ((-17005373.358 7893332.312, -17004122..."


In [10]:
# merge em
results_polys = results.merge(polys, how="left", on="GEOID")

In [11]:
# results for some places have a comma-separated string of places in the GEOID column
# so we need to explode these records, get the corresponding polygons, and dissolve those polygons in order to merge with results

dfs = []

for index_, row in results.iterrows():
    if "," in row["GEOID"]:
        geoids = row["GEOID"].split(", ")

        df = pd.DataFrame(data={"GEOID": geoids})
        df["id"] = row["id"]
        df["name"] = row["name"]
        df["GEOID"] = df["GEOID"].str[-9:]

        dfs.append(df)

multi = pd.concat(dfs)
multi_ = multi.merge(polys, how="left", on="GEOID")
multi_poly = gpd.GeoDataFrame(multi_, geometry="geometry")
dissolved = multi_poly.dissolve(by="id", as_index=False)

In [12]:
# merge the dissolved polygons and combine the geometry columns into one
gdf = results_polys.merge(dissolved[["id", "geometry"]], how="left", on="id")
gdf.loc[gdf["geometry_x"].isna(), "geometry_x"] = gdf["geometry_y"]
gdf.drop(columns="geometry_y", inplace=True)
gdf.rename(columns={"geometry_x": "geometry"}, inplace=True)
gdf = gpd.GeoDataFrame(gdf, geometry="geometry")

In [13]:
# check for missing geometry... only the state of AK and the US should be missing
gdf[gdf.geometry.isna()]

,id,name,areatype,placename,GEOID,total_population,pct_65_plus,pct_under_18,pct_under_5,pct_hispanic_latino,...,pct_single_parent,pct_unemployed,comment,pct_w_disability_low,pct_w_disability_high,pct_insured_low,pct_insured_high,pct_uninsured_low,pct_uninsured_high,geometry
421,AK0,Alaska,State,Alaska,02,733391.0,12.98,24.46,6.56,6.79,...,6.12,7.21,NaN,12.3,12.9,87.9,88.7,11.3,12.1,None
422,US0,United States,Nation,United States,1,331449281.0,16.83,22.06,5.55,18.73,...,6.56,5.64,NaN,12.8,13.0,91.2,91.4,8.6,8.8,None


In [14]:
# finally, replace the word "county" with "borough" in the comment column
gdf["comment"] = gdf["comment"].replace({"county": "borough"}, regex=True)
gdf

,id,name,areatype,placename,GEOID,total_population,pct_65_plus,pct_under_18,pct_under_5,pct_hispanic_latino,...,pct_single_parent,pct_unemployed,comment,pct_w_disability_low,pct_w_disability_high,pct_insured_low,pct_insured_high,pct_uninsured_low,pct_uninsured_high,geometry
0,BORO1,Aleutians East Borough,County,Aleutians East Borough,013,3420.0,6.87,8.77,2.05,19.71,...,7.90,4.50,NaN,8.30,12.50,56.40,64.20,35.80,43.60,"MULTIPOLYGON (((-18371741.628 7280122.270, -18..."
1,BORO19,Municipality of Anchorage,County,Anchorage Municipality,020,291247.0,12.41,23.46,6.43,9.08,...,6.00,5.40,NaN,10.70,11.70,88.80,90.20,9.80,11.20,"POLYGON ((-16745398.154 8664388.754, -16743113..."
2,AK15,Anchorage,County,Anchorage Municipality,020,291247.0,12.41,23.46,6.43,9.08,...,6.00,5.40,Data represent information from nearest boroug...,10.70,11.70,88.80,90.20,9.80,11.20,"POLYGON ((-16745398.154 8664388.754, -16743113..."
3,CENS9,Bethel Census Area,County,Bethel Census Area,050,18666.0,8.24,35.02,9.61,1.11,...,8.80,18.10,NaN,10.60,12.40,84.30,87.70,12.30,15.70,"MULTIPOLYGON (((-17838823.054 8815510.630, -17..."
4,BORO17,Bristol Bay Borough,County,Bristol Bay Borough,060,844.0,16.11,22.04,5.33,5.33,...,3.80,2.40,NaN,10.50,19.30,74.80,87.00,13.00,25.20,"POLYGON ((-17588474.981 8103444.343, -17586660..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420,AK435,Yakutat,Census designated place,Yakutat CDP,86490,657.0,18.42,19.18,4.72,4.26,...,2.30,1.60,Data represent information from nearest census...,6.20,13.00,64.70,88.90,11.10,35.30,"POLYGON ((-15569502.654 8298459.929, -15569487..."
421,AK0,Alaska,State,Alaska,02,733391.0,12.98,24.46,6.56,6.79,...,6.12,7.21,NaN,12.30,12.90,87.90,88.70,11.30,12.10,None
422,US0,United States,Nation,United States,1,331449281.0,16.83,22.06,5.55,18.73,...,6.56,5.64,NaN,12.80,13.00,91.20,91.40,8.60,8.80,None
423,AK103,Eagle River,Census tract,"Census Tract 2.01, Census Tract 2.02, Census T...","020000201, 020000202, 020000204, 020000205, 02...",25118.0,10.37,27.03,6.96,7.84,...,4.01,2.82,Data for this place represent multiple merged ...,6.75,14.95,90.74,97.76,2.24,9.26,"POLYGON ((-16654510.910 8694034.271, -16654430..."


In [15]:
pd.set_option("display.max_colwidth", None)
print(gdf[gdf["id"] == "AK418"][["id", "name", "comment"]])

        id       name  \
404  AK418  Utqiaġvik   

                                                                                                    comment  
404  Data represent information from nearest incorporated place (Utqiaġvik city), which includes Utqiaġvik.  


In [16]:
# export as shp, which will auto-truncate column names to 10 characters
# we will fix these columns names in the API after querying from GeoServer!
gdf.to_file("shp/demographics.shp", encoding="utf-8")

/var/folders/zs/cxjm36s175z0cdc0qbkhf5rh0000gp/T/ipykernel_655/1690917955.py:3: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file("shp/demographics.shp", encoding="utf-8")


In [17]:
# double check that the encoding is correct and preserves the dotted g in Utqiaġvik
saved_shp = gpd.read_file("shp/demographics.shp")

In [18]:
pd.set_option("display.max_colwidth", None)
print(saved_shp[saved_shp["id"] == "AK418"][["id", "name", "comment"]])

        id       name  \
404  AK418  Utqiaġvik   

                                                                                                    comment  
404  Data represent information from nearest incorporated place (Utqiaġvik city), which includes Utqiaġvik.  


In [19]:
# check out the truncated column names
saved_shp.columns

Index(['id', 'name', 'areatype', 'placename', 'GEOID', 'total_popu',
       'pct_65_plu', 'pct_under_', 'pct_unde_1', 'pct_hispan', 'pct_white',
       'pct_africa', 'pct_amer_i', 'pct_asian', 'pct_hawaii', 'pct_other',
       'pct_multi', 'pct_w_disa', 'pct_insure', 'pct_uninsu', 'pct_asthma',
       'pct_hd', 'pct_copd', 'pct_diabet', 'pct_emospt', 'pct_foodst',
       'pct_mh', 'pct_stroke', 'pct_asth_1', 'pct_hd_low', 'pct_copd_l',
       'pct_diab_1', 'pct_emos_1', 'pct_food_1', 'pct_mh_low', 'pct_stro_1',
       'pct_asth_2', 'pct_hd_hig', 'pct_copd_h', 'pct_diab_2', 'pct_emos_2',
       'pct_food_2', 'pct_mh_hig', 'pct_stro_2', 'pct_no_bba', 'pct_crowdi',
       'pct_hcost', 'pct_no_hsd', 'pct_below_', 'pct_minori', 'pct_single',
       'pct_unempl', 'comment', 'pct_w_di_1', 'pct_w_di_2', 'pct_insu_1',
       'pct_insu_2', 'pct_unin_1', 'pct_unin_2', 'geometry'],
      dtype='object')

In [20]:
# create a dict to map only the truncated column names to the column names we want in the API
demographics_fields = {
    "pct_65_plu": "pct_65_plus",
    "pct_africa": "pct_african_american",
    "pct_amer_i": "pct_amer_indian_ak_native",
    "pct_below_": "pct_below_150pov",
    "pct_diabet": "pct_diabetes",
    "pct_hawaii": "pct_hawaiian_pacislander",
    "pct_hispan": "pct_hispanic_latino",
    "pct_insure": "pct_insured",
    "pct_minori": "pct_minority",
    "pct_no_bba": "pct_no_bband",
    "pct_no_hsd": "pct_no_hsdiploma",
    "pct_under_": "pct_under_18",
    "pct_unde_1": "pct_under_5",
    "pct_uninsu": "pct_uninsured",
    "pct_w_disa": "pct_w_disability",
    "pct_foodst": "pct_foodstamps",
    "pct_crowdi": "pct_crowding",
    "pct_single": "pct_single_parent",
    "pct_unempl": "pct_unemployed",
    "total_popu": "total_population",
    # the fields below are low and high confidence intervals (95% CI values)
    "pct_asth_1": "pct_asthma_low",
    "pct_asth_2": "pct_asthma_high",
    "pct_copd_l": "pct_copd_low",
    "pct_copd_h": "pct_copd_high",
    "pct_diab_1": "pct_diabetes_low",
    "pct_diab_2": "pct_diabetes_high",
    "pct_emos_1": "pct_emospt_low",
    "pct_emos_2": "pct_emospt_high",
    "pct_food_1": "pct_foodstamps_low",
    "pct_food_2": "pct_foodstamps_high",
    "pct_mh_hig": "pct_mh_high",
    "pct_hd_hig": "pct_hd_high",
    "pct_stro_1": "pct_stroke_low",
    "pct_stro_2": "pct_stroke_high",
    'pct_w_di_1' : "pct_w_disability_low",
    'pct_w_di_2' : "pct_w_disability_high",
    'pct_insu_1' : "pct_insured_low",
    'pct_insu_2' : "pct_insured_high",
    'pct_unin_1' : "pct_uninsured_low",
    'pct_unin_2' : "pct_uninsured_high",
}